In [3]:
import torchvision
import torch
from torchvision import datasets, transforms
import numpy as np
# from tqdm import tqdm
from scipy.special import logsumexp
from multiprocessing import Pool

MAX_ITER = 10
EPSILON = 1e-7
TOLERANCE = 1e-5
transform = transforms.Compose([transforms.ToTensor()])

c:\Users\KandaMaya\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def initializeModel(K, d):
    np.random.seed(0)
    pi = np.random.rand(K)
    pi = pi / np.sum(pi)

    np.random.seed(0)
    mu = np.random.normal(0, 3, size=(K, d))

    np.random.seed(0)
    S = np.random.rand(K, d) + 0.5 + EPSILON

    return pi, mu, S



In [5]:
print(initializeModel(4, 3))
a, b, c = initializeModel(4,3)

(array([0.2275677 , 0.29655611, 0.24993822, 0.22593797]), array([[ 5.29215704,  1.20047163,  2.93621395],
       [ 6.7226796 ,  5.60267397, -2.93183364],
       [ 2.85026525, -0.45407162, -0.30965656],
       [ 1.23179551,  0.43213071,  4.36282052]]), array([[1.0488136 , 1.21518947, 1.10276348],
       [1.04488328, 0.9236549 , 1.14589421],
       [0.93758731, 1.3917731 , 1.46366286],
       [0.88344162, 1.29172514, 1.02889502]]))


In [6]:
def GMM(X, K_RANGE):
    N, d = X.shape
    pi, mu, S = initializeModel(K_RANGE, d)
    log_r = np.zeros((N, K_RANGE))
    loss = [0.0] * MAX_ITER

    for iter in range(MAX_ITER):
        print(pi)
        for k in range(K_RANGE):
            log_r[:,k] = np.log(pi[k] + EPSILON) - 0.5 * np.sum(np.log(S[k] + EPSILON)) - 0.5 * np.sum((X-mu[k]) ** 2 / (S[k] + EPSILON), axis = 1)
        
        log_r_i = logsumexp(log_r, axis = 1)
        log_r = log_r - log_r_i[:,None]
        loss[iter] = -np.sum(log_r_i)

        if iter > 1 and abs(loss[iter] - loss[iter-1]) <= TOLERANCE * abs(loss[iter]):
            break

        r = np.exp(log_r)
        r_dot_k = np.sum(r, axis = 0)
        pi = r_dot_k / N
        mu = np.matmul(r.T, X) / (r_dot_k[:,None] + EPSILON)
        S = np.matmul(r.T, X**2) / (r_dot_k[:,None] + EPSILON) - mu ** 2

    return pi, mu, S, loss

In [7]:
print("train for class {}".format(0))
data_train = datasets.MNIST(root = "./data/",
                        transform=transform,
                        train = True,
                        download = True)
idx = data_train.targets == 0
np_X = data_train.data[idx].numpy()
N, d1, d2 = np_X.shape
X = np_X.reshape(N, d1*d2)
pi, mu, S, loss = GMM(X, 5)
pis[k] = pi
mus[k] = mu
Ss[k] = S
losses[k] = [ val for val in loss if val > 0.0]

train for class 0
[0.19356424 0.25224431 0.21259213 0.19217803 0.14942128]
[0.021273   0.13709269 0.53503292 0.13337836 0.17322303]


C:\Users\KandaMaya\AppData\Local\Temp\ipykernel_8736\485548835.py:10: RuntimeWarning: invalid value encountered in log
  log_r[:,k] = np.log(pi[k] + EPSILON) - 0.5 * np.sum(np.log(S[k] + EPSILON)) - 0.5 * np.sum((X-mu[k]) ** 2 / (S[k] + EPSILON), axis = 1)


[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]


NameError: name 'pis' is not defined

In [29]:
data_train = datasets.MNIST(root = "./data/",
                        transform=transform,
                        train = True,
                        download = True)
idx = data_train.targets == 0
np_X = data_train.data[idx].numpy()
N, d1, d2 = np_X.shape
X = np_X.reshape(N, d1*d2) / 255.0
pi, mu, S, loss = GMM(X, 5)
print(pi)
print(mu)
print(S)
print(loss)

[9.00017103e-182 5.91626728e+003 1.49690782e-085 6.73272205e+000
 1.02583219e-240]
[1.00000000e+00 5.66800001e+03 1.00000000e+00 2.51999987e+02
 1.00000000e+00]
[1.00000000e+00 5.45607429e+03 1.00000000e+00 4.63925708e+02
 1.00000000e+00]
[1.000e+00 5.295e+03 1.000e+00 6.250e+02 1.000e+00]
[1.00000000e+00 5.11178031e+03 1.00000000e+00 8.08219686e+02
 1.00000000e+00]
[1.00000000e+00 5.00900001e+03 1.00000000e+00 9.10999995e+02
 1.00000000e+00]
[1.00000000e+00 4.85310371e+03 1.00000000e+00 1.06689629e+03
 1.00000000e+00]
[1.00000000e+00 4.73038144e+03 1.00000000e+00 1.18961856e+03
 1.00000000e+00]
[1.0000000e+00 4.6420225e+03 1.0000000e+00 1.2779775e+03 1.0000000e+00]
[1.00000000e+00 4.51710492e+03 1.00000000e+00 1.40289508e+03
 1.00000000e+00]
[1.68833361e-04 7.62638007e-01 1.68833361e-04 2.36855493e-01
 1.68833361e-04]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 

In [31]:
X = np.loadtxt(open("gmm_dataset.csv", "rb"), delimiter=",")

K = 10
pis, mus, Ss, losses = [[None for _ in range(K)] for _ in range(4)]
for k in range(10):
    pi, mu, S, loss = GMM(X, k+1)
    pis[k] = pi
    mus[k] = mu
    Ss[k] = S
    losses[k] = [ val for val in loss if val > 0.0]
    print(pi)


[5000.]
[5000.]
[1.]
[3256.54594257 1743.45405743]
[3258.94072729 1741.05927271]
[3122.96725161 1877.03274839]
[3063.23074802 1936.76925198]
[3226.93707645 1773.06292355]
[3443.90169029 1556.09830971]
[3500.39241273 1499.60758727]
[3503.04748491 1496.95251509]
[0.7006095 0.2993905]
[1574.46615022 1109.73922231 2315.79462746]
[1800.1174638   715.31482745 2484.56770875]
[1975.75873004  515.48918358 2508.75208638]
[1980.12390741  500.50850914 2519.36758346]
[1977.51925912  499.75124554 2522.72949534]
[0.39550385 0.09995025 0.5045459 ]
[ 704.57364746  453.48756566  933.44928314 2908.48950374]
[ 744.04617538  499.41485304  996.81954234 2759.71942925]
[ 828.12299587  499.48392452  999.10808988 2673.28498972]
[1070.75845199  499.29614962  999.300914   2430.64448439]
[1658.50724002  499.20872771  999.5367571  1842.74727517]
[1990.1921504   499.42653472 1000.61551511 1509.76579976]
[1999.12782984  499.65287239 1001.2294612  1499.98983657]
[1999.0670886   499.67069395 1001.27939102 1499.98282643

In [32]:
for loss in losses:
    print(loss[-1])

69939.41787313473
58105.13225849246
51670.13914769469
41554.07119920562
34512.117207307274
34347.080310634
34314.399090849474
34281.32037530586
34237.36920278936
34210.69281425627


In [38]:
pis, mus, Ss, losses = [[None for _ in range(10)] for _ in range(4)]

for k in range(10):
    for identify_cls in range(10):
        print("train for class {}".format(identify_cls))
        data_train = datasets.MNIST(root = "./data/",
                                transform=transform,
                                train = True,
                                download = True)
        idx = data_train.targets == identify_cls
        np_X = data_train.data[idx].numpy()
        N, d1, d2 = np_X.shape
        X = np_X.reshape(N, d1*d2) / 255.0
        pi, mu, S, loss = GMM(X, 5)
        pis[k] = pi
        mus[k] = mu
        Ss[k] = S
        losses[k] = [ val for val in loss if val > 0.0]

train for class 0
train for class 1
train for class 2
train for class 3
train for class 4
train for class 5
train for class 6
train for class 7
train for class 8
train for class 9
train for class 0
train for class 1
train for class 2
train for class 3
train for class 4
train for class 5
train for class 6
train for class 7
train for class 8
train for class 9
train for class 0
train for class 1
train for class 2
train for class 3
train for class 4
train for class 5
train for class 6
train for class 7
train for class 8
train for class 9
train for class 0
train for class 1
train for class 2
train for class 3
train for class 4
train for class 5
train for class 6
train for class 7
train for class 8
train for class 9
train for class 0
train for class 1
train for class 2
train for class 3
train for class 4
train for class 5
train for class 6
train for class 7
train for class 8
train for class 9
train for class 0
train for class 1
train for class 2
train for class 3
train for class 4
train for 

In [48]:

data_train = datasets.MNIST(root = "./data/",
                        transform=transform,
                        train = True,
                        download = True)
idx = data_train.targets == 0
np_X = data_train.data[idx].numpy()
N, d1, d2 = np_X.shape
X = np_X.reshape(N, d1*d2)
pi, mu, S, loss = GMM(X, 5)
print(pi)
print(mu)
print(S)
print(loss)

C:\Users\KandaMaya\AppData\Local\Temp\ipykernel_2384\557739713.py:14: RuntimeWarning: invalid value encountered in log
  log_r[:,k] = np.log(pi[k]) - 0.5 * np.sum(np.log(S[k])) - 0.5 * np.dot((X-mu[k]) ** 2, 1/S[k])


[nan nan nan nan nan]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[24574538708.09118, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]


In [37]:
X = np.array([[1,2,3],[4,5,6],[7,8,9],[10,11,12]])
mu = np.mean(X, axis=0)
print(mu)
print(X-mu)
np.sum((X - mu) ** 2, axis=0)

[5.5 6.5 7.5]
[[-4.5 -4.5 -4.5]
 [-1.5 -1.5 -1.5]
 [ 1.5  1.5  1.5]
 [ 4.5  4.5  4.5]]


array([45., 45., 45.])

In [38]:
X + mu

array([[ 6.5,  8.5, 10.5],
       [ 9.5, 11.5, 13.5],
       [12.5, 14.5, 16.5],
       [15.5, 17.5, 19.5]])

In [6]:
X = np.array([[1,2,3,4]])
X**2

array([[ 1,  4,  9, 16]])

In [ ]:
X = 